In [1]:
#!bash /content/drive/MyDrive/single-person-pose-estimation/dataset/get_data.sh

In [2]:
import time
from datetime import datetime, timedelta
import sys

In [3]:
sys.path.append('/content/drive/MyDrive/single-person-pose-estimation')

In [4]:
from configs import default_config as cfg
from coco_df import gen_trainval_df
from gen_tfrecords import gen_TFRecords

In [5]:
train_df, valid_df = gen_trainval_df(cfg)

loading annotations into memory...
Done (t=23.56s)
creating index...
index created!
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
Only examples that are not crowd and num_keypoints > 0 are chosen !
Length of train df: 149813
Length of valid df: 6352


In [6]:
start = time.time()
gen_TFRecords(train_df, cfg, is_train = True)
total_time = time.time() - start
print("Total time: {}".format(str(timedelta(seconds=total_time))))

TFRecords generated at dataset/tfrecords/train
Total time: 0:24:16.377075


In [7]:
start = time.time()
gen_TFRecords(valid_df, cfg, is_train = False)
total_time = time.time() - start
print("Total time: {}".format(str(timedelta(seconds=total_time))))

TFRecords generated at dataset/tfrecords/valid
Total time: 0:00:59.336691


In [8]:
#check size, good size per file ~100Mb
!ls -lh "dataset/tfrecords/train" 
!ls -lh "dataset/tfrecords/valid" 

total 5.6G
-rw-r--r-- 1 root root 77M Mar 17 10:00 file_train_00-2048.tfrec
-rw-r--r-- 1 root root 77M Mar 17 10:01 file_train_01-2048.tfrec
-rw-r--r-- 1 root root 77M Mar 17 10:01 file_train_02-2048.tfrec
-rw-r--r-- 1 root root 83M Mar 17 10:01 file_train_03-2048.tfrec
-rw-r--r-- 1 root root 80M Mar 17 10:02 file_train_04-2048.tfrec
-rw-r--r-- 1 root root 76M Mar 17 10:02 file_train_05-2048.tfrec
-rw-r--r-- 1 root root 77M Mar 17 10:02 file_train_06-2048.tfrec
-rw-r--r-- 1 root root 81M Mar 17 10:03 file_train_07-2048.tfrec
-rw-r--r-- 1 root root 76M Mar 17 10:03 file_train_08-2048.tfrec
-rw-r--r-- 1 root root 76M Mar 17 10:03 file_train_09-2048.tfrec
-rw-r--r-- 1 root root 74M Mar 17 10:04 file_train_10-2048.tfrec
-rw-r--r-- 1 root root 82M Mar 17 10:04 file_train_11-2048.tfrec
-rw-r--r-- 1 root root 76M Mar 17 10:04 file_train_12-2048.tfrec
-rw-r--r-- 1 root root 76M Mar 17 10:05 file_train_13-2048.tfrec
-rw-r--r-- 1 root root 76M Mar 17 10:05 file_train_14-2048.tfrec
-rw-r--r-- 1 r



---



---

Just testing

In [9]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [10]:
### For reading
def parse_tfrecord_fn(example):
    feature_description = {
        "ann_id": tf.io.FixedLenFeature([], tf.int64),
        "image_id": tf.io.FixedLenFeature([], tf.int64),
        "image": tf.io.FixedLenFeature([], tf.string),
        "image_path": tf.io.FixedLenFeature([], tf.string),
        "width": tf.io.FixedLenFeature([], tf.int64),
        "height": tf.io.FixedLenFeature([], tf.int64),
        "keypoints/x": tf.io.VarLenFeature(tf.float32),
        "keypoints/y": tf.io.VarLenFeature(tf.float32),
        "keypoints/vis": tf.io.VarLenFeature(tf.int64),
        "keypoints/num": tf.io.FixedLenFeature([], tf.int64)
    }
    
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.image.decode_image(example["image"], channels = 3, dtype = tf.float32, expand_animations =False)
    example["keypoints/x"] = tf.sparse.to_dense(example["keypoints/x"])
    example["keypoints/y"] = tf.sparse.to_dense(example["keypoints/y"])
    example["keypoints/vis"] = tf.sparse.to_dense(example["keypoints/vis"])
    return example
  ### example
def gen_examples_from_tfrecord(filepath, example_num):
  raw_dataset = tf.data.TFRecordDataset(filepath)
  parsed_dataset = raw_dataset.map(parse_tfrecord_fn)
  for example in parsed_dataset.take(example_num):
      for key in example.keys():
          if key != "image":
              print(f"{key}: {example[key]}")

      xcoords = example["keypoints/x"].numpy()
      ycoords = example["keypoints/y"].numpy()
      image = example["image"]
      h, w, c = image.shape
      print(f"Image shape: {image.shape}")
      plt.figure(figsize=(7, 7))
  
      plt.imshow(image)
      plt.scatter(xcoords  , ycoords  , marker = "o") # for heatmap size
      plt.show()  

In [12]:
gen_examples_from_tfrecord('dataset/tfrecords/valid/file_valid_00-2048.tfrec', 10)

Output hidden; open in https://colab.research.google.com to view.



---



---

IF run on local machine skip the following

In [13]:
!zip -r 'dataset/tfrecords.zip' 'dataset/tfrecords'

  adding: dataset/tfrecords/ (stored 0%)
  adding: dataset/tfrecords/valid/ (stored 0%)
  adding: dataset/tfrecords/valid/file_valid_03-208.tfrec (deflated 2%)
  adding: dataset/tfrecords/valid/file_valid_00-2048.tfrec (deflated 3%)
  adding: dataset/tfrecords/valid/file_valid_01-2048.tfrec (deflated 3%)
  adding: dataset/tfrecords/valid/file_valid_02-2048.tfrec (deflated 2%)
  adding: dataset/tfrecords/train/ (stored 0%)
  adding: dataset/tfrecords/train/file_train_23-2048.tfrec (deflated 2%)
  adding: dataset/tfrecords/train/file_train_54-2048.tfrec (deflated 2%)
  adding: dataset/tfrecords/train/file_train_09-2048.tfrec (deflated 2%)
  adding: dataset/tfrecords/train/file_train_33-2048.tfrec (deflated 2%)
  adding: dataset/tfrecords/train/file_train_28-2048.tfrec (deflated 2%)
  adding: dataset/tfrecords/train/file_train_29-2048.tfrec (deflated 2%)
  adding: dataset/tfrecords/train/file_train_38-2048.tfrec (deflated 2%)
  adding: dataset/tfrecords/train/file_train_55-2048.tfrec (def

In [14]:
!cp 'dataset/tfrecords.zip' 'drive/MyDrive/single-person-pose-estimation/dataset'